In [1]:
import os
import subprocess
import math
from pydub import AudioSegment
from dotenv import load_dotenv

load_dotenv(".env")

api_key = os.getenv('OPENAI_API_KEY')

def extract_audio_from_video(video_path, audio_path):
    command = ["wsl", "ffmpeg", "-i", video_path, "-vn", audio_path]
    subprocess.run(command)

"""
extract_audio_from_video("./files/video.mp4", "./files/video.mp3")
"""

def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size*60*1000 
    
    chunks = math.ceil(len(track) / chunk_len)

    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i +1) * chunk_len
        
        chunk = track[start_time:end_time]
        
        chunk.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")


In [6]:
import openai
import glob

#

#transcript["text"]

def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transcript = openai.Audio.transcribe("whisper-1", audio_file)
            text_file.write(transcript["text"])

transcribe_chunks("./files/chunks", "./files/transcript.txt")